# Gerrymandering and the Redistricting Problem

## Assumptions

We assume a 2-party system with the parties represented as "D" and "R". We start by only using one period's worth of historical data. We also assume that all available voters voted (no one abstained) in our historical data.

## Data Inputs

The expected number of votes for each party is a key data input for our constraints. We represent this with a matrix, $\mathbf{V}$. Since we have two parties, the matrix has the following shape.

$$ \mathbf{V} \in \mathbb{R}^{|blocks| \times 2} $$

$blocks$ is a set where $|blocks|$ is the number of elements in the set. By convention, the first column of $\mathbf{V}$ will be D votes and the second column R votes. $\mathbf{V}$ could represent a single past election's results, or it could be an expected upcoming result based on an exogenous model.

The second major input is a matrix specifying which blocks are contiguous with each other, meaning that they share a border. This matrix has the following shape:

$$ \mathbf{C} \in \mathbb{R}^{|blocks| \times |blocks|} $$

The elements of the matrix are defined as 

$$ c_{ij} = \left\{ \begin{array}{cc} 1 & \text{block i borders block j} \\ 0 & \text{otherwise} \end{array} \right. $$

## Objective 

Our objective uses the concept of the "efficiency gap". See [Brennan Center](https://www.brennancenter.org/sites/default/files/legal-work/How_the_Efficiency_Gap_Standard_Works.pdf). The efficiency gap itself uses another concept of "wasted votes". 

Votes can be wasted in two ways. A vote cast for a losing candidate is "wasted", and votes cast for a winning candidate in excess of the amount needed to win are also wasted. The efficiency gap is a signed number so it can be in favor of one party or another. We arbitrarily chose to do it from the perspective of the "D" party. So a positive efficiency gap favors the R party. 

$$ \text{Efficiency Gap} = \text{D wasted votes} - \text{R wasted votes} $$


## Variables

There are several variables in the model. The key variable is a matrix where each row of the matrix represents an indivisible block (precint, county, or census area depending on the conventions of the problem). The columns represent assignment to a district. The matrix is made up of zeroes or ones, and each row must have exactly one entry equal to one, meaning that each row must be in one and only one district. $districts$ is a set where $|districts|$ is the number of elements in each set. 

$$ 
\mathbf{D} \in \{0,1\}^{|blocks| \times |districts|}. 
$$

Several other variables are necessary to set up the problem in a linear fashion. These are best explained in the context of each constraint.

## Constraints

### Each Block Is In Exactly One District

This constraint is easily expressed by saying that the sum of each row in the $D$ variable must be exactly one. 

$$ D \left[ \begin{array}{c} 1 \\ \vdots \\ 1 \end{array} \right]  = \left[ \begin{array}{c} 1 \\ \vdots \\ 1 \end{array} \right].$$

### Calculate The Number of Wasted Votes for Losing Party

To calculate wasted votes for the losing party, we first have to know which party lost and what its vote total was. We need to know this for each district that is formed as a result of the optimization. A simple `min` function is not linear, and we desire a purely linear form of the problem. This can be achieved using the "Big M" method. See [Big M](https://en.wikipedia.org/wiki/Big_M_method). Suppose we are looking at the results in just one district, with vote totals $d$ and $r$. Then define two additional variables, $wastedUnder$ and $w$. Further choose a constant $M$ that is large enough. Then define two constraints as below.

$$ \begin{align}
wastedUnder &\geq d - Mw \\
wastedUnder &\geq r - M(1-w) \\
wastedUnder &\in \mathbb{R} \\
w &\in {0,1}
\end{align} $$

If we include $wastedUnder$ in the objective function to minimize it, then the optimizer will try to reduce it. If $d$ is smaller than $r$, it will minimize it by setting $w=0$, and allowing $wastedUnder = d$.  The other constraint is nonbinding in this case. Otherwise, it will set $w=1$, making the first constraint non-bonding and setting $wastedUnder = r$. Since $w$ must be either 0 or 1, then $wastedUnder$ will be the minimum. 

### Calculate Wasted Votes for the Winning Party

Wasted votes for the winning party occur when the winning party gets more votes than is necessary to win. It is mostly simply calculate as 
$$\max (0, \text{winning votes} - \text{threshhold to win} ).$$

Because we want to avoid `max` functions, which are non-linear, we use a trick similar to the one we used for wasted votes for the losing party. Let $VotesToWin$ be the threshold to win (50% plus one). Then set up constraints as follows.

$$
\begin{align}
wastedOver & \geq 0 \\
wastedOver & \geq d - VotesToWin 
\end{align}
$$

If we include $wastedOver$ in the objective to minimize it, then it will be $0$ if the $D$ party lost and $D - VotesToWin$ otherwise. This is the value we seek.

### Enforce Equal Sizes

Equal sizes are enforced by defining a single variable for the problem. Then we require that the total votes cast in each district be within a certain range around the single variable.

### Enforce Contiguity of Districts

The contiguity constraints requires a contiguity matrix, $C$. Let $D$ be the assignment of blocks to districts. Consider the matrix $CD$ with elements $a_{ij}$. Each element, $a_{ij}$, is then the number of blocks contiguous with block $i$ that are also in district $j$, provided that block $i$ is in district $j$.  If block $i$ is not in district $j$, then $a_{ij}$ is meaningless for us. If a block, $i$, is in a given district, then we require at least one other block that is contiguous with $i$ is also in the same district. We can write the constraint this way:

$$ \mathbf{CD} \geq 2 \times \mathbf{D}.$$

This says that, if block $i$ is in district $j$, then there must be at least one other block contiguous with block $i$ (in addition to block $i$ itself) that is also in district $j$.  Since every element of $CD$ is necessarily positive, an element $a_{ij}$ is effectively unconstrained if the corresponding element of $D$ is zero.

## Optimization Model Definition

We are now able to define a general purpose function to use with different data sets and parameters.

In [1]:
using JuMP 
using GLPKMathProgInterface
using Cbc
# using Gurobi
#using NLopt
#using AmplNLWriter
#using CoinOptServices
#using ECOS

In [13]:
function degerry(
    votes,
    contiguity_matrix, 
    number_districts,
    common_size_threshold = 0.2
    )
    
    _V = votes
    _C = contiguity_matrix

    blocks = size(_V,1)
    districts = number_districts
    total_vote = _V * ones(2,1)

    #m = Model(solver = AmplNLSolver(CoinOptServices.bonmin, [
    #    "bonmin.pump_for_minlp=yes"]) )
    # m = Model(solver = ECOSSolver() )
    #m = Model(solver = GLPKSolverMIP())
    m = Model(solver = CbcSolver())
    # m = Model(solver=NLoptSolver(algorithm=:LD_SLSQP))
    #m = Model(solver = GurobiSolver(Presolve=0))
    
    ## Variables

    @variable(m, 0 <= D[i=1:blocks,j=1:districts] <= 1 , Bin)
    
    ## Constraints

    # each block can be in only one district
    @constraint(m, D * ones(districts,1) .== 1)  
    
    # Each district must have at least one block
    # @constraint(m, (D' * V) * [1;1] .>= 1)

    # These constraints set wasted_u to the number of wasted votes for the losing party
    @variable(m, 0 <= w[i=1:districts] <= 1, Bin)
    @variable(m, wasted_u[i=1:districts, j=1:2])
    M = blocks * sum(total_vote) 
    @constraint(m, wasted_u .>= 0)
    @constraint(m, wasted_u[:,1] .>= (D' * _V)[:,1] - M * w)
    @constraint(m, wasted_u[:,2] .>= (D' * _V)[:,2] - M * (1-w))

    # These constraints set wasted_o to the number of wasted votes for the winning party
    @variable(m, wasted_o[i=1:districts, j=1:2])
    @variable(m, votes_to_win[i=1:districts])
    @constraint(m, votes_to_win .== (D' * _V) * [1;1] / 2)
    @constraint(m, wasted_o .>= 0)
    @constraint(m, wasted_o .>= (D' * _V) - votes_to_win * [1 1])

    # These constraints calculate the efficiency gap
    @variable(m, eff_gap)
    @variable(m, abs_eff_gap)
    @constraint(m, eff_gap .== ones(1,districts) * (wasted_u + wasted_o) * [1;-1])
    @constraint(m, abs_eff_gap >= eff_gap)
    @constraint(m, abs_eff_gap >= - eff_gap)

    # These constraints enforce roughly equal sizes. 
    @variable(m, common_size) # this approach is too slow
    fixed_common_size = sum(_V) / districts
    @constraint(m, (D' * _V) * [1;1] .>= fixed_common_size * (1-common_size_threshold))
    @constraint(m, (D' * _V) * [1;1] .<= fixed_common_size * (1+common_size_threshold))

    # These constraints enforce contiguity
    @constraint(m, _C * D .>= 2 * D)

    ## Objective

    @objective(m, Min, abs_eff_gap  + sum(wasted_u) + sum(wasted_o) ) #+ sum(min_vote)) 
    
    @time begin
        status = solve(m)
    end
    
    res = Dict([("Model",m),
        ("Solve Status", status), 
        ("Efficiency Gap", getvalue(abs_eff_gap) ),
        ("Wasted Over Votes", getvalue(wasted_o)),
        ("Wasted Under Votes", getvalue(wasted_u)),
        ("Total Wasted Votes [D R]", ones(1,districts) * ( getvalue(wasted_u) + getvalue(wasted_o))),
        ("Votes By District", getvalue(D)' * _V), 
        ("Common Size", getvalue(common_size)), 
        ("Fixed Common Size", fixed_common_size), 
        ("District Assignments", getvalue(D))
    ])
    
    return res
end

degerry (generic function with 2 methods)

## Example Problem

This is a simple problem used to show how the model works.

In [7]:
# These are the votes arranged in a matrix of size blocks x number of parties
V = [75 25; 60 40; 43 57; 48 52; 49 51]


5×2 Array{Int64,2}:
 75  25
 60  40
 43  57
 48  52
 49  51

In [8]:
# This is the contiguity matrix. C_{m,n} = 1 if block m shares a border with block n, 0 otherwise.
C = [ 
    1 1 1 0 0;
    1 1 1 1 0;
    1 1 1 1 1;
    0 1 1 1 1;
    0 0 1 1 1;]

## So the example looks like this:
## |----------|
## |A    | B  |
## |-----|    |
## |     |----|
## | C   |    |
## |     | D  |
## |     |----|
## |     | E  |
## |-----|----|


5×5 Array{Int64,2}:
 1  1  1  0  0
 1  1  1  1  0
 1  1  1  1  1
 0  1  1  1  1
 0  0  1  1  1

In [9]:
res = degerry(V,C, 2, 0.2)
res["District Assignments"]
res

  2.099823 seconds (104.55 k allocations: 5.788 MiB, 0.89% gc time)


Dict{String,Any} with 9 entries:
  "Efficiency Gap"           => -3.58993e-13
  "Solve Status"             => :Optimal
  "Wasted Over Votes"        => [0.0 3.0; 28.0 0.0]
  "Wasted Under Votes"       => [97.0 0.0; 0.0 122.0]
  "Model"                    => Minimization problem with:…
  "Total Wasted Votes [D R]" => [125.0 125.0]
  "Votes By District"        => [97.0 103.0; 178.0 122.0]
  "District Assignments"     => [0.0 1.0; 0.0 1.0; … ; 1.0 0.0; 1.0 0.0]
  "Common Size"              => 250.0

## Optimize Fairness With Realistic Data

TODO: insert optimization with Wisconsin data set



In [5]:
# Load data
using CSV
WI_votes = CSV.read("data/Gerrymander County_election_data.csv")
WI_contiguity = CSV.read("data/Gerrymander County_contiguity.csv", rows = 73)


INFO: Precompiling module CSV.


,County,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,78,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137,139,141,,src_COUNTY,nbr_COUNTY,contiguity_flag
1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,missing,1,1,1
2,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,missing,1,21,1
3,5,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,missing,1,57,1
4,7,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,missing,1,77,1
5,9,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,missing,1,97,1
6,11,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,missing,1,111,1
7,13,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,missing,1,137,1
8,15,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,missing,1,141,1
9,17,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,missing,3,3,1
10,19,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,missing,3,7,1


In [6]:
WI_V = convert(Array, WI_votes[:,3:4])
WI_C = convert(Array, WI_contiguity[:,2:73])

72×72 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0  1  0  0  …  0  0  0  0  0  0  0  0  0  1  0  1
 0  1  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  1  0  1  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  1  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  …  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  1  0  0  1  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  1  0  0  0  0  0  1  1  0  0  0     1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  1  0  0  0     1  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  1  0  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  1    

In [7]:
# check that the contiguity matrix is symmetric
all(WI_C == WI_C')

true

In [16]:
res3 = degerry(WI_V,WI_C, 3, 0.5)

145.359214 seconds (85 allocations: 1.134 MiB)


Dict{String,Any} with 10 entries:
  "Efficiency Gap"           => -6.71231e-9
  "Solve Status"             => :Optimal
  "Wasted Over Votes"        => [27523.5 0.0; 1.81143e5 0.0; 0.0 1257.0]
  "Wasted Under Votes"       => [0.0 338090.0; 0.0 395554.0; 526235.0 0.0]
  "Model"                    => Minimization problem with:…
  "Total Wasted Votes [D R]" => [734901.0 734901.0]
  "Votes By District"        => [393137.0 338090.0; 757839.0 395554.0; 526235.0…
  "Fixed Common Size"        => 979868.0
  "Common Size"              => 0.0
  "District Assignments"     => [1.0 0.0 0.0; 0.0 1.0 0.0; … ; 0.0 0.0 1.0; 0.0…

In [17]:
res4 = degerry(WI_V,WI_C, 4, 0.5)

 97.980101 seconds (86 allocations: 1.512 MiB)


Dict{String,Any} with 10 entries:
  "Efficiency Gap"           => 2.91666e-9
  "Solve Status"             => :Optimal
  "Wasted Over Votes"        => [50617.5 0.0; 0.0 13342.5; 0.0 3576.5; 1.7371e5…
  "Wasted Under Votes"       => [0.0 361387.0; 295394.0 0.0; 215179.0 0.0; 0.0 …
  "Model"                    => Minimization problem with:…
  "Total Wasted Votes [D R]" => [734901.0 734901.0]
  "Votes By District"        => [462622.0 361387.0; 295394.0 322079.0; 215179.0…
  "Fixed Common Size"        => 734901.0
  "Common Size"              => 0.0
  "District Assignments"     => [0.0 0.0 0.0 1.0; 0.0 0.0 0.0 1.0; … ; 1.0 0.0 …

In [18]:
res5 = degerry(WI_V,WI_C, 5, 0.5)

 51.531381 seconds (86 allocations: 1.887 MiB)


Dict{String,Any} with 10 entries:
  "Efficiency Gap"           => 6.50786e-9
  "Solve Status"             => :Optimal
  "Wasted Over Votes"        => [91284.0 0.0; 41265.5 0.0; … ; 98295.5 0.0; 0.0…
  "Wasted Under Votes"       => [0.0 204255.0; 0.0 272468.0; … ; 0.0 234742.0; …
  "Model"                    => Minimization problem with:…
  "Total Wasted Votes [D R]" => [734901.0 734901.0]
  "Votes By District"        => [386823.0 204255.0; 354999.0 272468.0; … ; 4313…
  "Fixed Common Size"        => 5.87921e5
  "Common Size"              => 0.0
  "District Assignments"     => [0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 …

In [19]:
res6 = degerry(WI_V,WI_C, 6, 0.5)

451.251259 seconds (89 allocations: 2.260 MiB)


Dict{String,Any} with 10 entries:
  "Efficiency Gap"           => -1.46532e-8
  "Solve Status"             => :Optimal
  "Wasted Over Votes"        => [1.01165e5 0.0; 42568.0 0.0; … ; 0.0 387.0; 115…
  "Wasted Under Votes"       => [0.0 266211.0; 0.0 181238.0; … ; 163738.0 0.0; …
  "Model"                    => Minimization problem with:…
  "Total Wasted Votes [D R]" => [734901.0 734901.0]
  "Votes By District"        => [468540.0 266211.0; 266374.0 181238.0; … ; 1637…
  "Fixed Common Size"        => 489934.0
  "Common Size"              => 0.0
  "District Assignments"     => [0.0 0.0 … 1.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 …